# Temporal Conversions

In [1]:
import pystare
import datetime
import numpy
import erfa
import astropy

## Conversions

In [2]:
import re

regex_iso8601 = r'^(-?(?:[1-9][0-9]*)?[0-9]{4})-(1[0-2]|0[1-9])-(3[01]|0[1-9]|[12][0-9])T(2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9])(\.[0-9]+)?(Z|[+-](?:2[0-3]|[01][0-9]):[0-5][0-9])?$'
match_iso8601 = re.compile(regex_iso8601).match


regex_stare = r'^(-?(?:[1-9][0-9]*)?[0-9]{4})-(1[0-2]|0[1-9])-(3[01]|0[1-9]|[12][0-9])T(2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9])\.([0-9]+)?(\s\(([0-9]+)\s([0-9]+)\)\s\(([0-9])\))$'
match_stare = re.compile(regex_stare).match


def validate_iso8601(str_val):    
    if match_iso8601(str_val) is not None:
        return True
    else:
        return False
    
def validate_stare_timestring(str_val):
    if match_stare(str_val) is not None:
        return True
    else:
        return False
    
    
def stare_timestring(timestamp, forward_res=48, backwards_res=48, stare_type=1, time_standard='UTC'):
    if isinstance(timestamp, datetime.datetime):
        time_string = timestamp.isoformat()
    elif isinstance(timestamp, numpy.datetime64):
        time_string = numpy.datetime_as_string(timestamp)
    elif isinstance(timestamp, tuple) and len(timestamp) == 2:
        # SOFA/ERFA tuple        
        time_string = erfa.d2dtf(scale=time_standard, ndp=6,  d1=timestamp[0], d2=timestamp[1])
    elif isinstance(timestamp, str) and validate_iso8601(timestamp):
        # ISO 8601 timestamp
        time_string = timestamp
    elif isinstance(timestamp, str) and validate_stare_timestamp(timestamp):
        # Already stare timestamp
        return timestamp
    else:
        raise ValueError('unknown input "' + timestamp + '"')
    suffix = ' ({f_res}) ({b_res}) ({type})'.format(f_res=forward_res, b_res=backwards_res, type=stare_type)
    return time_string + suffix

        
def erfajd_from_timestamp(timestamp, time_scale='UTC'):
    if isinstance(timestamp, numpy.datetime64):
        timestamp = timestamp.astype(datetime.datetime)
    elif isinstance(timestamp, str) and validate_iso8601(timestamp):
        timestamp = datetime.datetime.fromisoformat(timestamp)
    
    if isinstance(timestamp, datetime.datetime):        
        erfa_jd = erfa.dtf2d(scale=time_scale, 
                             iy=timestamp.year, im=timestamp.month, id=timestamp.day, 
                             ihr=timestamp.hour, imn=timestamp.minute, sec=timestamp.second+timestamp.microsecond/1000/1000)
    
    return erfa_jd

def stare_from_timestamp(timestamp, forward_res=48, backwards_res=48, stare_type=1, time_scale='UTC'):
    stare_ts = stare_timestring(timestamp, forward_res, backwards_res, staret_type, time_scale)
    pystare.from_tai_iso_string(stare_ts)

In [3]:
ts = '2021-01-09T17:47:56.154564 (45 12) (1)'
validate_iso8601('2021-01-09T17:47:56.154564 (45 12) (1)')
validate_iso8601('2021-01-09T17:47:56.2435+05:00')
match_iso8601(ts)

### UTC MS coversions

In [4]:
now = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
now = numpy.array([now], dtype='datetime64[ms]')
now = now.astype(numpy.int64)
now

array([1629999406000])

In [5]:
timestamps = numpy.array(['2021-01-03'], dtype='datetime64[ms]').astype(numpy.int64)
pystare.from_ms_since_epoch_utc(timestamps, 48, 48)

array([2275448110396223681])

In [6]:
tivs = [2275448110396223681]
ts = pystare.to_ms_since_epoch_utc(tivs).astype('datetime64[ms]')
numpy.datetime_as_string(ts)

array(['2021-01-03T00:00:00.000'], dtype='<U42')

In [7]:
ts1 = numpy.array(['2021-01-03T01', '1985-01-03T01'], dtype='datetime64[ms]').astype(numpy.int64)
ts2 = numpy.array(['2021-05-01T10', '1986-10-01'], dtype='datetime64[ms]').astype(numpy.int64)
tiv1 = pystare.from_ms_since_epoch_utc(ts1, 10, 10)
tiv2 = pystare.from_ms_since_epoch_utc(ts2, 10, 10)
pystare.cmp_temporal(tiv1, tiv2, flatten=False)

array([[1, 0],
       [0, 1]])

### Resolutions

In [8]:
millisecond  = 1
second = 1000 * millisecond
minute = 60 * second
hour = 60 * minute
day = 86400 * second
year = 365 * day
times = numpy.array([millisecond, second, minute, hour, day, year], dtype=numpy.int64)
pystare.coarsest_resolution_finer_or_equal_ms(times)

array([48, 38, 32, 26, 21, 12])

### Iso Strings

In [9]:
def pad3(s):
    s = s.ljust(4, '0')
    s=s[0:4]
    return s

s = '2021-08-26T17:03:56.6'
ms = match_iso8601(s).groups()[6]
s.replace(ms, pad3(ms))

'2021-08-26T17:03:56.600'

In [10]:
time_strings = ['2021-08-26T17:03:56.6']
pystare.from_iso_strings(time_strings, forward_res=20, reverse_res=18)

array([2276038620409631817])

In [21]:
time_strings = ['2021-08-26T17:03:56.626 (48 48) (1)']
tiv = pystare.from_stare_timestrings(time_strings, scale='TAI')
tiv

array([2276038620410065089])

In [23]:
pystare.to_stare_timestring(tiv) == time_strings

True

In [40]:
pystare.to_stare_timestring(tiv)

['2021-08-26T17:03:56.626 (48 48) (1)']

### JDs

In [13]:
now = datetime.datetime.now()
jd1 = erfajd_from_timestamp(now)
jd2 = erfajd_from_timestamp(now.isoformat())
jd3 = erfajd_from_timestamp(numpy.datetime64(now))
jd1 == jd2 == jd3

True

In [14]:
erfa.jdcalf(3, *jd1)[-1]/1000*24

17.616

In [36]:
now.isoformat()

'2021-08-26T17:36:46.426092'

In [37]:
from astropy import time
t = time.Time(now.isoformat(), format='isot')
t.jd1, t.jd2

(2459453.0, 0.23387067236111114)

In [41]:
pystare.to_julian_tai(tiv)

(array([2459215.5]), array([237.71107206]))

In [42]:
tiv

array([2276038620410065089])

### Temporal Triple

In [24]:
triple = pystare.to_temporal_triple_tai(tiv)
triple = pystare.to_temporal_triple_ms(tiv)
triple

(array([2276038620410036477]),
 array([2276038620410065089]),
 array([2276038620410085373]))

In [25]:
pystare.from_temporal_triple(numpy.array(triple).flatten())

array([2276038620410065089])

### Julian TAI

In [26]:
time = datetime.datetime(2021,2,1,12,0)
time_tuple = time.timetuple()

time = erfa.dtf2d("UTC", *time_tuple[0:6])
erfa.jd2cal(*time)

(2021, 2, 1, 0.5)

In [30]:
pystare.from_julian_tai(numpy.array([time[0]]), 
                       numpy.array([time[1]]))

array([2275521502176608257])

In [31]:
pystare.from_julian_utc(numpy.array([time[0]]), 
                       numpy.array([time[1]]))

array([2275521502797365249])

In [32]:
t = pystare.to_JulianTAI(tiv)
t = pystare.to_JulianUTC(tiv)
t

AttributeError: module 'pystare' has no attribute 'to_JulianTAI'

## Bounds

In [ ]:
pystare.lowerBoundTAI
pystare.upperBoundTAI
pystare.lowerBoundMS
pystare.upperBoundMS

## Resolution 

In [ ]:
set_reverse_resolution
set_forward_resolution
reverse_resolution
forward_resolution
coarsen
set_temporal_resolutions_from_sorted
coarsest_resolution_finer_or_equal_ms

## Comparison

In [ ]:
cmp_temporal

temporalValueIntersectionIfOverlap
temporalValueUnionIfOverlap
temporalOverlapTAI
temporalOverlap
temporalContainsInstant